In [9]:
import os
import pickle

import numpy as np
import pandas as pd
import wavespectra 
import xarray as xr

DATA_DIRECTORY = '/vortexfs1/home/csherwood/proj/NOPP/buoy_data/'
DATA_FILENAME = 'hurricane_ian_spotter_data_v1.pickle'

with open(os.path.join(DATA_DIRECTORY, DATA_FILENAME), 'rb') as handle:
    spotter = pickle.load(handle)

# `spotter` is a python dictionary of Pandas DataFrames, keyed by
# each drifter ID. The drifter ids can then be accessed as follows:
spotter_ids = list(spotter.keys())

# Dictionary to store wavespectra datasets in; can be any container.
wavespectra_datasets = {}

for spotter_id in spotter.keys():

    # Extract the observation times that contain spectral data.
    only_waves = spotter[spotter_id]['energy_density'].notnull()
    drifter = spotter[spotter_id][only_waves]

    # Exract the coordinate arrays; note that the frequency array is
    # uniform across the Spotter observations, so we can just 
    # use the array in the first index of the DataFrame.
    time = drifter.index.to_numpy()
    freq = drifter['frequency'][0] 

    # Extract the variable arrays.
    efth = np.stack(drifter['energy_density'])
    lat = drifter['latitude']
    lon = drifter['longitude']

    # Construct the dataset. This must match the conventions used by the
    # wavespectra package:
    # (https://wavespectra.readthedocs.io/en/latest/conventions.html#)
    # Note that the directional spectrum needs to be computed using the
    # directional moments and an estimator (e.g. MEM). As shown here, 
    # the dataset will be constructed, but it will be scalar, e.g. 
    # efth(time, freq) and not efth(time, freq, dir).
    ds = xr.Dataset(
        data_vars=dict(
            # efth=(["freq", "dir", "time"], efth), #TODO: need to compute this
            efth=(["time", "freq"], efth), # Delete this line if using above (Note: efth may need to be transposed)
            lat=(["time"], lat),
            lon=(["time"], lon),
            site="",
        ),
        coords=dict(
            time=time,
            freq=freq,
            dir=[], #TODO: need to compute this from directional moments
        ),
        attrs=dict(
            # Attributes here; wavespectra would put significant wave
            # height, etc., here but these are a function of time so it
            # might be reasonable to set them as data_vars instead (as 
            # a function of the time coordinate)
        )
    )

    # Format and attach SpecArray accessor to the existing xarray 
    # dataset using the wavespectra.read_dataset() method. Store it in 
    # the dictionary keyed by spotter_id.
    wavespectra_datasets[spotter_id] = wavespectra.read_dataset(ds)
    

In [20]:
spec2d = estimate( drifter.a1[0], 

array([ 0.032258, -0.045943, -0.043988,  0.01173 , -0.032258, -0.047898,
       -0.081134, -0.162268, -0.12219 , -0.158358, -0.088954, -0.087977,
       -0.043988, -0.071359,  0.01173 , -0.046921, -0.182796, -0.12219 ,
       -0.202346, -0.284457, -0.378299, -0.519062, -0.575758, -0.575758,
       -0.589443, -0.544477, -0.57869 , -0.539589, -0.532747, -0.532747,
       -0.577713, -0.59824 , -0.606061, -0.617791, -0.577713, -0.602151,
       -0.58651 , -0.538612, -0.369501])

In [17]:
wavespectra_datasets[spotter_ids[0]].spec.stats(['hs','tp'])

<xarray.Dataset>
Dimensions:  (time: 72)
Coordinates:
  * time     (time) object 2022-09-27T00:35:55+00:00 ... 2022-09-29T23:35:55+...
Data variables:
    hs       (time) float64 0.5956 0.6214 0.5817 0.6008 ... 2.275 2.269 2.269
    tp       (time) float32 dask.array<chunksize=(72,), meta=np.ndarray>
Attributes:
    standard_name:  sea_surface_wave_significant_height
    units:          m